In [15]:
from dotenv import load_dotenv
load_dotenv()
import os

In [16]:
import nest_asyncio
nest_asyncio.apply()

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Load the data
Here, we will download the 2023 IPPC Climate Report - Chapter 3 on Oceans and Coastal Ecosystems (172 Pages)

In [ ]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 37 20.7M   37 7949k    0     0  6503k      0  0:00:03  0:00:01  0:00:02 6527k
 82 20.7M   82 17.0M    0     0  7855k      0  0:00:02  0:00:02 --:--:-- 7872k
100 20.7M  100 20.7M    0     0  7436k      0  0:00:02  0:00:02 --:--:-- 7448k


In [14]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./attention.pdf"]
).load_data()

In [18]:
len(documents)

15

In [19]:
# %pip install llama-index-llms-openai
# %pip install llama-index-extractors-entity

# Setup the LLM that is going to be used for extraction

In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode
llm = OpenAI(temperature=0.1, model="gpt-4o-mini", max_tokens=512)

# Setup the Extractor and Parser


In [3]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import TokenTextSplitter


c:\Code\Github\LlamaIndex\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Code\Github\LlamaIndex\venv\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in EntityExtractor has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Setup text splitter

In [4]:
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

# Define Custom metadata Extractor using the Base extractor

In [9]:
class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list

# Create the list of extractors (inclusing predefined and custom extractor)

In [11]:
extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
    EntityExtractor(prediction_threshold=0.5),
    SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]

# Define the transformations 

In [12]:
transformations = [text_splitter] + extractors

In [20]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

nodes_with_metadata = pipeline.run(documents=documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Provided proper attribution is provided, Google...
> Adding chunk: Provided proper attribution is provided, Google...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: days on eight GPUs, a small fraction of the tra...
> Adding chunk: days on eight GPUs, a small fraction of the tra...


DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 1 Introduction
Recurrent neural networks, long ...
> Adding chunk: 1 Introduction
Recurrent neural networks, long ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: goal of reducing sequential computation also fo...
> Adding chunk: goal of reducing sequential computation also fo...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Figure 1: The Transformer - model architecture....
> Adding chunk: Figure 1: The Transformer - model architecture....
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Scaled Dot-Product Attention
 Multi-Head Attent...
> Adding chunk: Scaled Dot-Product Attention
 Multi-Head Attent...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: where it has
extremely small gradients4. To cou...
> Adding chunk: where it has
extremely small gradients4. To cou...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: output values. These are concatenated 

  0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Context: days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature. We show that the Transformer generalizes well to\nother tasks by applying it successfully to English constituency parsing both with\nlarge and limited training data.\n∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started\nthe effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and\nh

 50%|█████     | 1/2 [00:01<00:01,  1.43s/it]

...

 93%|█████████▎| 28/30 [00:08<00:00,  3.42it/s]

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 02 Nov 2024 09:00:24 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-f8xnqrzkrmt0qbpgbtsecvpc'), (b'openai-processing-ms', b'552'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9943'), (b'x-ratelimit-remaining-tokens', b'182062'), (b'x-ratelimit-reset-requests', b'8m4.65s'), (b'x-ratelimit-reset-tokens', b'5.381s'), (b'x-request-id', b'req_1d8a1675e05c6b78083cb2395e61b8d5'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8dc2ef1a9a259194-DEL'), (b'Content-Encoding',

 97%|█████████▋| 29/30 [00:08<00:00,  3.55it/s]

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 02 Nov 2024 09:00:25 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-f8xnqrzkrmt0qbpgbtsecvpc'), (b'openai-processing-ms', b'1229'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9945'), (b'x-ratelimit-remaining-tokens', b'183696'), (b'x-ratelimit-reset-requests', b'7m47.935s'), (b'x-ratelimit-reset-tokens', b'4.89s'), (b'x-request-id', b'req_227de320d439546fbc97e663e0c46d77'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8dc2ef170bff8e80-DEL'), (b'Content-Encoding

100%|██████████| 30/30 [00:08<00:00,  3.44it/s]


In [21]:
nodes_with_metadata

[TextNode(id_='6b1d6dee-2d36-4692-8734-b6bed6f2fd6b', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': 'attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-11-02', 'last_modified_date': '2024-11-02', 'document_title': '"Revolutionizing Natural Language Processing: The Impact and Innovations of Transformer Architecture in Sequence Transduction and Machine Translation"', 'questions_this_excerpt_can_answer': 'Based on the provided excerpt from the document, here are three specific questions that can be answered using the context:\n\n1. **What is the main innovation introduced by the authors in the paper titled "Attention Is All You Need"?**\n   - The authors propose a new network architecture called the Transformer, which is based solely on attention mechanisms and eliminates the need for recurrence and convolutions.\n\n2. **What were the performance results of the Transformer model on the WMT 2014 English-to-

In [22]:
len(nodes_with_metadata)

30

# Try a Query!


In [23]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.2)

index_with_metadata = VectorStoreIndex(nodes=nodes_with_metadata)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000222A546C3A0>, 'json_data': {'input': ['[Excerpt from document] page_label: 1 file_path: attention.pdf document_title: "Revolutionizing Natural Language Processing: The Impact and Innovations of Transformer Architecture in Sequence Transduction and Machine Translation" questions_this_excerpt_can_answer: Based on the provided excerpt from the document, here are three specific questions that can be answered using the context:  1. **What is the main innovation introduced by the authors in th

In [24]:
index_with_metadata.storage_context.persist(persist_dir="./index_with_metadata")

DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/index_with_metadata/docstore.json
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/index_with_metadata/docstore.json
DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/index_with_metadata/index_store.json
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/index_with_metadata/index_store.json
DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filter

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./index_with_metadata")

# Load the index
index = load_index_from_storage(storage_context)

In [26]:
query_engine = index_with_metadata.as_query_engine()
response = query_engine.query("What complexity, sequential operation and max path len of layer type convolutional?")


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000222A53F2950>, 'json_data': {'input': ['What complexity, sequential operation and max path len of layer type convolutional?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000222A53F2950>, 'json_data': {'input': ['What complexity, sequential operation and max path len of layer type convolutional?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/embeddings
Sending HTTP Request: POST https://api.openai.com/v1/embeddings
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.starte

In [27]:
print(response)

For the convolutional layer type, the complexity per layer is O(k·n·d²), the minimum number of sequential operations is O(1), and the maximum path length is O(logₖ(n)).


# Contrast without metadata
Here, we re-construct the index, but without metadata

In [28]:
from llama_index.core import SimpleDirectoryReader

documents_without_metadata = SimpleDirectoryReader(
    input_files=["./attention.pdf"]
).load_data()

DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/attention.pdf
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Extracting Metadata for Better Document Indexing and Understanding/attention.pdf


In [29]:
documents_without_metadata

[Document(id_='c3bcddb7-853f-493b-9910-e22625b9b472', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': 'attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-11-02', 'last_modified_date': '2024-11-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@go

In [35]:
from llama_index.core import VectorStoreIndex

index_without_metadata=VectorStoreIndex.from_documents(documents=documents,
    use_async=True,
    show_progress=True,
)


Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Provided proper attribution is provided, Google...
> Adding chunk: Provided proper attribution is provided, Google...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Attention Visualizations
Input-Input Layer5
It
...
> Adding chunk: Attention Visualizations
Input-Input Layer5
It
...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [5]Kyunghyun Cho, Bart van Merrienboer, Caglar ...
> Adding chunk: [5]Kyunghyun Cho, Bart van Merrienboer, Caglar ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Table 4: The Transformer generalizes well to En...
> Adding chunk: Table 4: The Transformer generalizes well to En...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: output values. These are concatenated and once ...
> Adding chunk: output values. These are concatenated and once ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Input-Input Layer5
The
Law
will
never


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function AsyncEmbeddings.create.<locals>.parser at 0x000001A1512ADCF0>, 'json_data': {'input': ['page_label: 1 file_path: attention.pdf  Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Resea

Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


In [37]:
index_without_metadata.storage_context.persist(persist_dir="./index_without_metadata")

DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Entity Metadata Extraction/index_without_metadata/docstore.json
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Entity Metadata Extraction/index_without_metadata/docstore.json
DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Entity Metadata Extraction/index_without_metadata/index_store.json
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Entity Metadata Extraction/index_without_metadata/index_store.json
DEBUG:fsspec.local:open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Metadata Extraction and Filters/Entity Metadata Extraction/index_without_metadata/graph_store.json
open file: c:/Code/Github/LlamaIndex/06.Advanced_Topics/2.Basic Strategies/Meta

# Load the Index (from storage) -- if needed

 When you need to use the index again, instead of re-indexing, you can load it from the persisted storage using load_index_from_storage.

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./index_without_metadata")

# Load the index
index_without_metadata = load_index_from_storage(storage_context)

In [ ]:
index_without_metadata.

In [ ]:
query_engine_without_metadata = index_without_metadata.as_query_engine()
response = query_engine.query("What complexity, sequential operation and max path len of layer type convolutional?")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001A151738820>, 'json_data': {'input': ['What complexity, sequential operation and max path len of layer type convolutional?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001A151738820>, 'json_data': {'input': ['What complexity, sequential operation and max path len of layer type convolutional?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/embeddings
Sending HTTP Request: POST https://api.openai.com/v1/embeddings
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.starte

In [40]:
print(response)


For the convolutional layer type, the complexity per layer is O(k·n·d²), the minimum number of sequential operations required is O(1), and the maximum path length is O(logk(n)).
